# PARA DISNEYLAND STUDIOS PARIS

In [11]:
import requests
import csv
import time
from datetime import datetime

In [12]:
def get_queu_information(api_url, output_file):
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Check if the request was successful
        data = response.json()  # Convert the response to JSON format

        with open(output_file, 'a', newline='') as csvfile:
            fieldnames = ['land_id', 'land_name', 'ride_id', 'ride_name', 'is_open', 'wait_time', 'last_update']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if csvfile.tell() == 0:
                writer.writeheader()


            for land in data.get('lands', []):
                land_id = land.get('id', 'N/A')
                land_name = land.get('name', 'N/A')

                for ride in land.get('rides', []):
                    ride_id = ride.get('id', 'N/A')
                    ride_name = ride.get('name', 'N/A')
                    is_open = ride.get('is_open', 'N/A')
                    wait_time = ride.get('wait_time', 'N/A')
                    last_update = ride.get('last_updated', 'N/A')

                    writer.writerow({'land_id': land_id, 'land_name': land_name, 'ride_id': ride_id,
                                     'ride_name': ride_name, 'is_open': is_open, 'wait_time': wait_time,
                                     'last_update': last_update})

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from the API: {e}")


In [16]:
total_hours = 2
interval_minutes = 30

# Directorio y nombre del archivo
output_directory = '/home/lucia/Documentos/themeparks/'
base_filename = 'queutimes_dlpStudios_'

# Bucle principal
for hour in range(total_hours):
    current_time = datetime.now()
    formatted_date = current_time.strftime('%Y%m%d')
    output_file = f"{output_directory}{base_filename}{formatted_date}.csv"

    # URL de la API
    api_url = "https://queue-times.com/parks/28/queue_times.json"

    # Obtener datos y guardar en el archivo correspondiente
    get_queu_information(api_url, output_file)

    # Esperar el intervalo antes de la siguiente solicitud
    time.sleep(interval_minutes * 60)

print("Bucle completado.")


Bucle completado.


# PARA TODOS LOS PARQUES

In [6]:
import json
# Ruta del archivo JSON que contiene los parques y las claves
PATH_JSON = "/home/lucia/Documentos/themeparks/list_themparks/parks_id_name.json"

# URL base de la API
URL_BASE = "https://queue-times.com/parks/{}/queue_times.json"
FILE_BASE = "database_{}.json"

def create_json_api_urls_file(json_file_path=PATH_JSON, api_url_base=URL_BASE, file_base=FILE_BASE):
    """
    Crea una lista de URLs basadas en las claves de un archivo JSON de parques.

    Args:
        json_file_path (str): Ruta del archivo JSON que contiene los parques y las claves.
        api_url_base (str): URL base de la API donde se inserta el ID del parque.
    
    Returns:
        List: Una lista de URLs formateadas con los ID de los parques.
    """
    try:
        # Cargar el archivo JSON
        with open(json_file_path, mode='r', encoding='utf-8') as json_file:
            parks_dict = json.load(json_file)
            url_name_dict = {}

            # Crear las URLs con las claves del diccionario
            for park_id, park_name in parks_dict.items():
                api_url = api_url_base.format(park_id)  # Formatear la URL con el ID del parque

                park_name_cleaned = park_name.replace(" ", "").lower()
                file_name = file_base.format(park_name_cleaned)

                url_name_dict[api_url] = file_name

            return url_name_dict

    except FileNotFoundError:
        print(f"El archivo JSON no se encontró: {json_file_path}")
    except json.JSONDecodeError as e:
        print(f"Error al leer el archivo JSON: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")


In [7]:
dict = create_json_api_urls_file()
dict


{'https://queue-times.com/parks/57/queue_times.json': "database_california'sgreatamerica.json",
 'https://queue-times.com/parks/58/queue_times.json': "database_canada'swonderland.json",
 'https://queue-times.com/parks/59/queue_times.json': 'database_carowinds.json',
 'https://queue-times.com/parks/50/queue_times.json': 'database_cedarpoint.json',
 'https://queue-times.com/parks/69/queue_times.json': 'database_dorneypark.json',
 'https://queue-times.com/parks/62/queue_times.json': 'database_kingsdominion.json',
 'https://queue-times.com/parks/60/queue_times.json': 'database_kingsisland.json',
 'https://queue-times.com/parks/61/queue_times.json': "database_knott'sberryfarm.json",
 'https://queue-times.com/parks/70/queue_times.json': "database_michigan'sadventure.json",
 'https://queue-times.com/parks/68/queue_times.json': 'database_valleyfair.json',
 'https://queue-times.com/parks/63/queue_times.json': 'database_worldsoffun.json',
 'https://queue-times.com/parks/331/queue_times.json': 'd

In [8]:
%%writefile script_urls.py
import json
import logging

# Configuración del logging
log_file = "script_urls.log"
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Ruta del archivo JSON que contiene los parques y las claves
PATH_JSON = "/home/lucia/Documentos/themeparks/list_themparks/parks_id_name.json"

# URL base de la API
URL_BASE = "https://queue-times.com/parks/{}/queue_times.json"
FILE_BASE = "database_{}.json"
OUTPUT_JSON = "url_routes.json"

def create_json_api_urls_file(json_file_path=PATH_JSON, 
                              api_url_base=URL_BASE, 
                              file_base=FILE_BASE, 
                              output_json=OUTPUT_JSON):
    """
    Crea una lista de URLs basadas en las claves de un archivo JSON de parques y guarda el diccionario en un archivo JSON.

    Args:
        json_file_path (str): Ruta del archivo JSON que contiene los parques y las claves.
        api_url_base (str): URL base de la API donde se inserta el ID del parque.
    
    Returns:
        dict: Un diccionario con las URLs como claves y los nombres de los archivos como valores.
    """
    try:
        # Cargar el archivo JSON
        with open(json_file_path, mode='r', encoding='utf-8') as json_file:
            parks_dict = json.load(json_file)
            url_name_dict = {}

            # Crear las URLs con las claves del diccionario
            for park_id, park_name in parks_dict.items():
                api_url = api_url_base.format(park_id)  # Formatear la URL con el ID del parque

                # Limpiar el nombre del parque (quitar espacios y convertir a minúsculas)
                park_name_cleaned = park_name.replace(" ", "").lower()
                file_name = file_base.format(park_name_cleaned)

                url_name_dict[api_url] = file_name

            # Guardar el diccionario como un archivo JSON
            with open(output_json, mode='w', encoding='utf-8') as output_file:
                json.dump(url_name_dict, output_file, indent=4, ensure_ascii=False)
            
            # Log de éxito
            logging.info(f"Archivo JSON 'url_routes.json' creado con éxito.")

    except FileNotFoundError:
        logging.error(f"El archivo JSON no se encontró: {json_file_path}")
    except json.JSONDecodeError as e:
        logging.error(f"Error al leer el archivo JSON: {e}")
    except Exception as e:
        logging.error(f"Error inesperado: {e}")

# Script principal
if __name__ == "__main__":
    create_json_api_urls_file()


Writing script_urls.py


In [11]:
%%writefile get_queu_information.py
import requests
import csv
import json
from datetime import datetime
import os
import logging

# Configuración del logging
log_file = "queu_information_parks.log"
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

OUTPUT_JSON = "url_routes.json"
output_directory = '/home/lucia/Documentos/themeparks/'

def get_queu_information(api_url, output_file):
    """
    Obtiene la información de la API y guarda los datos en un archivo CSV.

    Args:
        api_url (str): URL de la API que proporciona la información de los parques.
        output_file (str): Ruta donde se guardará el archivo CSV con la información.
    """
    try:
        logging.info(f"Solicitando datos de la API: {api_url}")
        response = requests.get(api_url)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa
        data = response.json()  # Convierte la respuesta a formato JSON

        with open(output_file, 'a', newline='') as csvfile:
            fieldnames = ['land_id', 'land_name', 'ride_id', 'ride_name', 'is_open', 'wait_time', 'last_update']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Escribe la cabecera solo si el archivo está vacío
            if csvfile.tell() == 0:
                writer.writeheader()

            for land in data.get('lands', []):
                land_id = land.get('id', 'N/A')
                land_name = land.get('name', 'N/A')

                for ride in land.get('rides', []):
                    ride_id = ride.get('id', 'N/A')
                    ride_name = ride.get('name', 'N/A')
                    is_open = ride.get('is_open', 'N/A')
                    wait_time = ride.get('wait_time', 'N/A')
                    last_update = ride.get('last_updated', 'N/A')

                    writer.writerow({'land_id': land_id, 'land_name': land_name, 'ride_id': ride_id,
                                     'ride_name': ride_name, 'is_open': is_open, 'wait_time': wait_time,
                                     'last_update': last_update})

        logging.info(f"Datos guardados correctamente en el archivo: {output_file}")

    except requests.exceptions.RequestException as e:
        logging.error(f"Error al obtener datos de la API: {e}")

with open(OUTPUT_JSON, mode='r', encoding='utf-8') as json_file:
    try:
        urls_file_name = json.load(json_file)
        logging.info(f"Archivo JSON '{OUTPUT_JSON}' cargado exitosamente.")
    except json.JSONDecodeError as e:
        logging.error(f"Error al leer el archivo JSON: {e}")
        raise
    except FileNotFoundError as e:
        logging.error(f"Archivo JSON no encontrado: {e}")
        raise
    except Exception as e:
        logging.error(f"Error inesperado al leer el archivo JSON: {e}")
        raise

for url_api, file_name in urls_file_name.items():
    # Crear el directorio correspondiente al nombre del parque si no existe
    directory_path = os.path.join(output_directory, file_name)
    os.makedirs(directory_path, exist_ok=True)
    logging.info(f"Directorio creado o ya existente: {directory_path}")

    # Crear el nombre base para el archivo de salida
    base_filename = f'queutimes_{file_name}_'
    current_time = datetime.now()
    formatted_date = current_time.strftime('%Y%m%d')
    output_file = os.path.join(directory_path, f"{base_filename}{formatted_date}.csv")

    logging.info(f"Creando archivo de salida: {output_file}")

    get_queu_information(url_api, output_file)

logging.info("Proceso completado.")


Writing get_queu_information.py
